In [1]:
import numpy as np
from math import log2

def calculate_entropy(labels):
    """Calculate the entropy of a set of labels."""
    # Count the occurrences of each label
    label_counts = {}
    total_samples = len(labels)
    for label in labels:
        label_counts[label] = label_counts.get(label, 0) + 1
    
    # Calculate entropy using the formula: -p_i * log2(p_i)
    entropy = 0
    for count in label_counts.values():
        probability = count / total_samples
        entropy -= probability * log2(probability)
    return entropy

def calculate_information_gain(feature_values, labels):
    """Calculate the information gain for a feature."""
    total_samples = len(labels)
    parent_entropy = calculate_entropy(labels)
    unique_values = set(feature_values)
    weighted_entropy = 0
    
    # Calculate the entropy of each subset based on unique feature values
    for value in unique_values:
        subset_indices = [i for i, val in enumerate(feature_values) if val == value]
        subset_labels = [labels[i] for i in subset_indices]
        subset_entropy = calculate_entropy(subset_labels)
        subset_weight = len(subset_indices) / total_samples
        weighted_entropy += subset_weight * subset_entropy
    
    # Calculate information gain as the difference between parent entropy and weighted entropy
    information_gain = parent_entropy - weighted_entropy
    return information_gain

def select_root_node(features, labels):
    """Select the root node using Information Gain."""
    num_features = len(features[0])
    information_gains = []
    
    # Calculate information gain for each feature and select the one with maximum gain
    for i in range(num_features):
        feature_values = [sample[i] for sample in features]
        information_gain = calculate_information_gain(feature_values, labels)
        information_gains.append(information_gain)
    
    # Select the index of the feature with maximum information gain
    root_node_index = np.argmax(information_gains)
    return root_node_index

features = [[1, 0], [1, 1], [0, 0], [0, 1]]
labels = [1, 1, 0, 0]
root_node_index = select_root_node(features, labels)
print("Index of selected root node:", root_node_index)


Index of selected root node: 0


In [2]:
import numpy as np
from collections import Counter

def equal_width_binning(feature_values, num_bins):
    """Perform equal width binning for continuous-valued features."""
    # Calculate the bin width based on the range of feature values
    min_value = min(feature_values)
    max_value = max(feature_values)
    bin_width = (max_value - min_value) / num_bins
    
    # Assign each feature value to the corresponding bin
    binned_values = [int((value - min_value) // bin_width) for value in feature_values]
    return binned_values

def frequency_binning(feature_values, num_bins):
    """Perform frequency binning for continuous-valued features."""
    # Count the occurrences of each feature value
    value_counts = Counter(feature_values)
    
    # Calculate bin counts based on the frequency of feature values
    bin_counts = {value: i // (len(value_counts) // num_bins) for i, value in enumerate(value_counts.keys())}
    
    # Assign each feature value to the corresponding bin
    binned_values = [bin_counts[value] for value in feature_values]
    return binned_values

def bin_continuous_feature(feature_values, num_bins, binning_type='equal_width'):
    """Bin continuous-valued features using the specified binning type."""
    if binning_type == 'equal_width':
        return equal_width_binning(feature_values, num_bins)
    elif binning_type == 'frequency':
        return frequency_binning(feature_values, num_bins)
    else:
        raise ValueError("Invalid binning type. Choose either 'equal_width' or 'frequency'.")

feature_values = [2.5, 3.5, 5.0, 6.5, 7.2, 8.0]
num_bins = 3
binning_type = 'equal_width'
binned_values = bin_continuous_feature(feature_values, num_bins, binning_type)
print("Equal Width Binned Values:", binned_values)


Equal Width Binned Values: [0, 0, 1, 2, 2, 3]


In [6]:
import numpy as np

class Node:
    def __init__(self, feat=None, split=None, left=None, right=None, leaf=None):
        """Initialize a tree node."""
        self.f = feat    # Feature index for splitting
        self.s = split   # Split value for continuous features
        self.l = left    # Left child node
        self.r = right   # Right child node
        self.leaf = leaf # Leaf node value (class label if leaf)
    def print_tree(self, depth=0):
        """Print the tree structure recursively."""
        indent = "  " * depth
        if self.leaf is not None:
            print(indent + "Leaf: Class", self.leaf)
        else:
            print(indent + "Feature", self.f, "<=", self.s)
            if self.l:
                self.l.print_tree(depth + 1)
            if self.r:
                self.r.print_tree(depth + 1)
def entropy(labels):
    """Calculate entropy of labels."""
    counts = {lbl: labels.count(lbl) for lbl in labels}  # Count occurrences of each label
    total = len(labels)                                  # Total number of labels
    entropy = sum(-count/total * np.log2(count/total) for count in counts.values())  # Calculate entropy
    return entropy

def best_split(features, labels):
    """Find best feature and split value."""
    best_gain = 0       # Initialize best information gain
    best_feat = None    # Initialize best feature index
    best_split = None   # Initialize best split value
    for i in range(len(features[0])):  # Iterate over each feature
        values = [sample[i] for sample in features]  # Get feature values
        unique = sorted(set(values))   # Get unique feature values
        for j in range(len(unique) - 1):  # Iterate over unique feature values for possible splits
            split = (unique[j] + unique[j+1]) / 2  # Calculate split value as average of adjacent unique values
            left_lbls = [labels[k] for k in range(len(labels)) if features[k][i] <= split]  # Labels in left split
            right_lbls = [labels[k] for k in range(len(labels)) if features[k][i] > split]   # Labels in right split
            # Calculate information gain
            gain = entropy(labels) - (len(left_lbls)/len(labels)) * entropy(left_lbls) - (len(right_lbls)/len(labels)) * entropy(right_lbls)
            # Update best gain, feature index, and split value if current gain is higher
            if gain > best_gain:
                best_gain = gain
                best_feat = i
                best_split = split
    return best_feat, best_split

def build_tree(features, labels, max_depth=None):
    """Build decision tree recursively."""
    # Base case: if maximum depth is reached or all labels are the same
    if max_depth is not None and (max_depth == 0 or len(set(labels)) == 1):
        return Node(leaf=max(labels, key=labels.count))  # Create leaf node with most common label
    feat, split = best_split(features, labels)  # Find best feature and split value
    # If no best split is found (e.g., due to categorical features with the same values), create leaf node
    if feat is None:
        return Node(leaf=max(labels, key=labels.count))
    # Partition data based on best split
    left_feats = [s for s in features if s[feat] <= split]  # Features in left split
    left_lbls = [labels[i] for i in range(len(labels)) if features[i][feat] <= split]  # Labels in left split
    right_feats = [s for s in features if s[feat] > split]  # Features in right split
    right_lbls = [labels[i] for i in range(len(labels)) if features[i][feat] > split]  # Labels in right split
    # Recursively build left and right subtrees
    left = build_tree(left_feats, left_lbls, max_depth - 1 if max_depth is not None else None)
    right = build_tree(right_feats, right_lbls, max_depth - 1 if max_depth is not None else None)
    return Node(feat=feat, split=split, left=left, right=right)  # Create internal node with best split

features = [[5, 1], [3, 1], [8, 2], [2, 2], [6, 3], [7, 3]]  
labels = [0, 0, 1, 1, 1, 0] 
tree = build_tree(features, labels)  
tree.print_tree()


Feature 1 <= 1.5
  Leaf: Class 0
  Feature 0 <= 6.5
    Leaf: Class 1
    Feature 0 <= 7.5
      Leaf: Class 0
      Leaf: Class 1
